In [22]:
import pandas as pd
import numpy as np

In [23]:
df = pd.read_csv(r"C:\Users\faisa\Desktop\DataSciencePurwadhika\Modul3ML\hotel_bookings.csv")
df.shape

(119390, 32)

In [25]:
len(df[['is_canceled', 'adr']].query("is_canceled == 0 & adr == 0")) / len(df) * 100

1.4624340397018176

Insight:
- percentage of pesanan yang tidak dibatalkan tapi adr = 0, karena hanya 1% maka dilakukan drop untuk analisa selanjutnya

In [26]:
df.query("is_canceled == 0 & adr == 0")

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
125,Resort Hotel,0,32,2015,July,27,4,0,1,4,...,No Deposit,NaN,NaN,0,Transient,0.0,0,1,Check-Out,2015-07-05
167,Resort Hotel,0,111,2015,July,28,6,0,0,2,...,No Deposit,240.0,NaN,0,Transient,0.0,0,2,Check-Out,2015-07-06
168,Resort Hotel,0,0,2015,July,28,6,0,0,1,...,No Deposit,250.0,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118631,City Hotel,0,78,2017,August,34,23,0,0,1,...,No Deposit,98.0,NaN,0,Transient-Party,0.0,0,0,Check-Out,2017-08-23
118762,City Hotel,0,1,2017,August,34,24,0,1,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,3,Check-Out,2017-08-25
118963,City Hotel,0,1,2017,August,35,27,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2017-08-27
119102,City Hotel,0,518,2017,August,34,26,2,1,1,...,No Deposit,229.0,NaN,0,Transient-Party,0.0,0,0,Check-Out,2017-08-29


In [27]:
# DataFrame tanpa is_canceled = 0 (tidak dibatalkan) and adr = 0 
data = df.query("~(is_canceled == 0 & adr == 0)")
data.shape

(117644, 32)

In [39]:
df.query("is_canceled == 1 & adr != 0")

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
8,Resort Hotel,1,85,2015,July,27,1,0,3,2,...,No Deposit,240.0,NaN,0,Transient,82.0,0,1,Canceled,2015-05-06
9,Resort Hotel,1,75,2015,July,27,1,0,3,2,...,No Deposit,15.0,NaN,0,Transient,105.5,0,0,Canceled,2015-04-22
10,Resort Hotel,1,23,2015,July,27,1,0,4,2,...,No Deposit,240.0,NaN,0,Transient,123.0,0,0,Canceled,2015-06-23
27,Resort Hotel,1,60,2015,July,27,1,2,5,2,...,No Deposit,240.0,NaN,0,Transient,107.0,0,2,Canceled,2015-05-11
32,Resort Hotel,1,96,2015,July,27,1,2,8,2,...,No Deposit,NaN,NaN,0,Transient,108.3,0,2,Canceled,2015-05-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108829,City Hotel,1,25,2017,May,18,6,2,1,1,...,No Deposit,NaN,450.0,0,Transient,125.0,0,0,No-Show,2017-05-06
111355,City Hotel,1,4,2017,June,23,5,1,0,1,...,No Deposit,NaN,238.0,0,Transient,65.0,0,0,Canceled,2017-06-05
111924,City Hotel,1,7,2017,May,22,31,0,1,1,...,No Deposit,NaN,238.0,0,Transient,65.0,0,0,Canceled,2017-05-31
111925,City Hotel,1,6,2017,July,29,17,1,0,1,...,No Deposit,NaN,238.0,0,Transient,65.0,0,0,No-Show,2017-07-17


In [28]:
# cek duplicated data
print(f"""jumlah data duplikat {data.query("@data.duplicated()").shape[0]} baris""")

# drop duplicated data
data = data.drop_duplicates()
data.duplicated().sum()

jumlah data duplikat 31841 baris


0

<h1><center> <span style="color:#0ad0f7">Revenue Analysis per Room</span></center></h1>

In [41]:
rev_data = data['adr'].to_frame()

# 5% of adr per room (rule of thumb marketing cost = 5% x revenue (adr))
rev_data['cost_marketing_per_room'] = 0.2 * rev_data['adr']
rev_data

,adr,cost_marketing_per_room
2,75.00,15.000
3,75.00,15.000
4,98.00,19.600
6,107.00,21.400
7,103.00,20.600
...,...,...
119385,96.14,19.228
119386,225.43,45.086
119387,157.71,31.542
119388,104.40,20.880


In [42]:
# rata-rata marketing cost
rev_data['cost_marketing_per_room'].mean()

21.662296166800697

Insight:
- rata-rata biaya marketing per room adalah **21.66 USD**

In [43]:
# rata-rata revenue tanpa marketing
rev_data['adr'].mean()

108.31148083400348

In [45]:
# rata-rata revenue dengan marketing
rev_data['adr'].mean() - 21.66

86.65148083400348